In [1]:
import faiss
import json
import openai
import numpy as np
from IPython.display import display, Markdown
from dotenv import load_dotenv
import os

load_dotenv()

API_KEY = os.environ.get("OPENAI_API_KEY")

# Set your OpenAI API Key
openai.api_key = API_KEY

# Define paths
faiss_index_file = "faiss_index.bin"
metadata_file = "metadata.json"

# Load FAISS index
embedding_dim = 1536  # OpenAI text-embedding-ada-002 output size
index = faiss.IndexFlatL2(embedding_dim)
index = faiss.read_index(faiss_index_file)

# Load metadata
with open(metadata_file, "r", encoding="utf-8") as f:
    metadata = json.load(f)

print("FAISS index and metadata loaded successfully! ✅")

FAISS index and metadata loaded successfully! ✅


In [19]:
# Function to generate embeddings using OpenAI
def get_embedding(text):
    response = openai.Embedding.create(
        model="text-embedding-ada-002",
        input=text
    )
    return np.array(response["data"][0]["embedding"], dtype=np.float32)

# Function to search FAISS and retrieve relevant text
def search_faiss(query, top_k=5):
    query_embedding = get_embedding(query)
    query_vector = np.array([query_embedding], dtype=np.float32)

    # Perform similarity search
    distances, indices = index.search(query_vector, top_k)

    results = []
    for idx in indices[0]:
        if str(idx) in metadata:
            results.append(metadata[str(idx)]["text"])

    return results

# Function to ask OpenAI GPT with retrieved context
def ask_openai(query, top_k=5, model="gpt-4o-mini"):
    # Retrieve relevant text
    retrieved_chunks = search_faiss(query, top_k)

    if not retrieved_chunks:
        return "No relevant context found."

    # Format retrieved text
    context = "\n\n".join(retrieved_chunks)
    print("CONTEXT:", context)
    # Query OpenAI GPT with retrieved context
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant answering questions based on a book."},
            {"role": "user", "content": f"Here is some context:\n{context}\n\nAnswer my question: {query}"}
        ]
    )

    return response["choices"][0]["message"]["content"]


In [27]:
query = "Hayri fırından ne alır?"  # Example question
answer = ask_openai(query)

# Display results in Markdown format (Jupyter Notebook)
display(Markdown(f"### ❓ **Question:** {query}"))
display(Markdown(f"### 💡 **Answer:** {answer}"))

CONTEXT: damlaları kendini belli etmeye başlıyordu: Hayri'nin terledigini fark eden babası elini alnına koyup: (6 'Hayri hasta mısın? Atesin mi çıktı acaba; terlemeye basladın:" dedi. Hayri bunu firsat bilerek: Akzun 182/ Hayal Odası rüya düşün- gece geçti. koyup göz tabağını Hayri: çok çok dinç diye gün hep Hay- hep 3gd,

gibi kendi etrafinda bir tur atıp: Hayri, geldi. gibi annesi çantadan Hayri'nin tarını yere yerine yıkadı. pek mıydım? yaşında gülünce geçme! şurada:"

feldiklerinde Hayri; kulağinı ve annesi bu yüzdene misafirleri ge- gitmişti. Babası Hayri sürük- teyze arkasını teyze. diye teyze an- Hayri diye katip Ahmet Ahmet Evin kapısının önüne kapıya içeriyi lecekti dinledi. Bugün alışverişe Tugba Akgun I44| Hayal Odasi

"Sen bakma onlara; gerçekten sunumun Bir gün toplanalım; bana şu hayalini ayrıntılı anlat. mu?" "Olur; tabii.. 1) dedi Hayri. Ödevine birilerinin vermesi hoşuna gitmişti. Sercan'ın yaptığı fedakar- Iık da gözünden kaçmamıştı. Yol boyu şakalaşıp gülü- şerek ile

### ❓ **Question:** Hayri fırından ne alır?

### 💡 **Answer:** Hayri, fırından almak için "bomba kurabiyeleri" alır. Her biri evde aileleriyle yemek için beşer tane bomba kurabiye almıştır.